In [44]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import matplotlib.pyplot as plt

from pathlib import Path
import os
import sys

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "9"

import torch
from torch import nn
from torch import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, QuantileTransformer

from pathlib import Path
with open("../default_paths.txt", 'r') as file:
    lines = file.readlines()
    data_path    = Path(lines[0].strip().split("=")[1])
    project_path = Path(lines[1].strip().split("=")[1])
    
utils_path = project_path / "utils"
train_path = data_path / "train"
test_path  = data_path / "test"
sys.path.insert(0, str(utils_path))

from data_utils_lesha import (process_all_files, process_string, save_sentences_to_txt, \
load_sentences_from_txt, WhisperSuperBrainDataset, count_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cuda')

In [2]:
""" LOAD PRETRAINED MODEL COMPONENTS """

WHISPER_MODEL_NAME = "openai/whisper-large-v3"

from transformers import WhisperTokenizer, WhisperFeatureExtractor
from transformers import GenerationConfig
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# import LORA modules
from peft import LoraConfig
from peft import LoraModel, AdaLoraModel, PeftModel, get_peft_model

from audiomentations import Compose, AddGaussianNoise, TimeStretch, LowPassFilter

# load feature/label processing engines
feature_extractor = WhisperFeatureExtractor.from_pretrained(WHISPER_MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(WHISPER_MODEL_NAME, task="transcribe")
# load model
model = WhisperForConditionalGeneration.from_pretrained(WHISPER_MODEL_NAME)

if ".en" not in WHISPER_MODEL_NAME:
    model.generation_config.language = "english"
    model.generation_config.task = "transcribe"
    model.generation_config.forced_decoder_ids = None

from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
metric = evaluate.load("wer")

from torchsummary import summary

# set a new convolutional layer to accept different number of input channels

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
#?Seq2SeqTrainingArguments

In [47]:
# CONV
new_conv = nn.Conv1d(512, model.model.encoder.conv1.out_channels, kernel_size=(3,), stride=(1,), padding=(1,))
model.model.encoder.conv1 = new_conv


# LORA
target_modules = ["k_proj", "q_proj", "v_proj", "out_proj", "fc1", "fc2"]

lora_config = LoraConfig(r=8, 
                         target_modules=target_modules, 
                         task_type="SEQ_2_SEQ_LM", 
                         use_rslora=True, 
                         use_dora=True)

model.model.decoder = LoraModel(model.model.decoder, lora_config, "default")


# print(summary(model))
# model

In [10]:
WHISPER_MODEL_NAME = "openai/whisper-large-v3"

experiment_path = data_path / "experiments" / WHISPER_MODEL_NAME / "decoder-DORA-r8-qkvo_augment-pad-less-filter"
checkpoint_path = experiment_path / "checkpoint-5206"


import safetensors
weights1 = safetensors.torch.load_file(checkpoint_path / "model-00001-of-00002.safetensors")
weights2 = safetensors.torch.load_file(checkpoint_path / "model-00002-of-00002.safetensors")
weights = {**weights1, **weights2}

#model = model.from_pretrained(checkpoint_path).to('cuda')

In [12]:
model.load_state_dict(weights, strict=False)

_IncompatibleKeys(missing_keys=['proj_out.weight'], unexpected_keys=[])

In [48]:
model.to(device);

In [49]:
%%time
data_train = process_all_files(train_path)
data_test = process_all_files(test_path)

XV = np.concatenate(data_train['voltage_list'], axis=0)
# scalerV = RobustScaler(quantile_range=(0.01, 0.99), unit_variance=True).fit(XV)
#scalerV = QuantileTransformer(n_quantiles=3000, output_distribution='uniform', subsample=30000).fit(XV)
scalerV = StandardScaler().fit(XV)

XS = np.concatenate(data_train['spike_list'], axis=0)
scalerS = MinMaxScaler(feature_range=(-1, 1)).fit(XS)

CPU times: user 20.3 s, sys: 7.46 s, total: 27.8 s
Wall time: 30.4 s


In [50]:
data_test['sentence_list'][0]

'theocracy reconsidered'

In [14]:
augment_train = Compose([
    LowPassFilter(10, 25, p=1, zero_phase=True),
    TimeStretch(min_rate=0.92, max_rate=1.08, p=0.5, leave_length_unchanged=False),
    AddGaussianNoise(min_amplitude=0.0001, max_amplitude=0.05, p=0.5),
])

augment_eval = Compose([LowPassFilter(15, 15.1, p=1, zero_phase=True)])

In [15]:
data_test = process_all_files(test_path)

eval_dataset = WhisperSuperBrainDataset(data_test['voltage_list'], 
                                         data_test['spike_list'], 
                                         data_test['sentence_list'], 
                                         tokenizer,
                                         scalerV, 
                                         scalerS, 
                                         augment_train, 
                                         augment_eval, 
                                         is_eval=True)

Scaling, concatenating, resampling the dataset...: 100%|██████████| 880/880 [00:08<00:00, 103.88it/s]


In [23]:
?model.generate

Signature:
model.generate(
    input_features: Optional[torch.Tensor] = None,
    generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None,
    logits_processor: Optional[transformers.generation.logits_process.LogitsProcessorList] = None,
    stopping_criteria: Optional[transformers.generation.stopping_criteria.StoppingCriteriaList] = None,
    prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
    synced_gpus: bool = False,
    return_timestamps: Optional[bool] = None,
    task: Optional[str] = None,
    language: Optional[str] = None,
    is_multilingual: Optional[bool] = None,
    prompt_ids: Optional[torch.Tensor] = None,
    prompt_condition_type: Optional[str] = None,
    condition_on_prev_tokens: Optional[bool] = None,
    temperature: Union[float, Tuple[float, ...], NoneType] = None,
    compression_ratio_threshold: Optional[float] = None,
    logprob_threshold: Optional[float] = None,
    no_speech_thres

In [29]:
for idx in range(0, 10):
    true_text = data_test['sentence_list'][idx]
    print(f"True: {true_text}")
    input_tensor = eval_dataset[idx]['input_features'].to('cuda').type(torch.float).reshape((1, 512, 3000))
    ids = model.generate(input_tensor, num_beam_groups=5, num_beams=20, diversity_penalty=0.01).cpu()
    pred_text = tokenizer.decode(ids[0], skip_special_tokens=True)
    print(f"Pred: {pred_text}\n")

True: Theocracy reconsidered.
Pred: The gun's good, he said.

True: Rich purchased several signed lithographs.
Pred: Touch points that have hidden visual effects.

True: So rules we made, in unabashed collusion.
Pred: The oil was put in animal oil.

True: Lori's costume needed black gloves to be completely elegant.
Pred: Roy kept playing much longer to be the official artiste.

True: The tooth fairy forgot to come when Roger's tooth fell out.
Pred: The other player will go to home and return to the field.

True: That stinging vapor was caused by chloride vaporization.
Pred: The king's best friend was called the lion, but he didn't say his name.

True: Before Thursday's exam, review every formula.
Pred: But now that they've gone, I'm well enough familiar.

True: Wildfire near Sunshine forces park closures.
Pred: What do you do when you're bored to death?

True: The word means it won't boil away easily, nothing else.
Pred: The old man ate one ball of oil every other day.

True: Would a b

In [42]:
p = 0.2
np.random.choice([0, 1], p=[1-p, p], size=10)

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 1])

In [43]:
data_train['sentence_list']

['Nuclear rockets can destroy airfields with ease.',
 'The best way to learn is to solve extra problems.',
 'The spray will be used in first division matches next season.',
 "Our experiment's positive outcome was unexpected.",
 "Alimony harms a divorced man's wealth.",
 'She uses both names interchangeably.',
 'The misquote was retracted with an apology.',
 'Critics fear the erosion of consumer protections and environmental standards.',
 'Her lips, moist and parted, spoke his name.',
 'How do they turn out later?',
 'The eastern coast is a place for pure pleasure and excitement.',
 'Please make your decision wisely to visit the Beach.',
 'She encouraged her children to make their own Halloween costumes.',
 'Cleaned cloth must be protected against the redeposition of dispersed soil.',
 'Both figures would go higher in later years.',
 'She always jokes about too much garlic in his food.',
 'If people were more generous, there would be no need for welfare.',
 'The dimensions of these wave